# PO-240: Pré-Processamento

### Importando Bibliotecas

In [1]:
from datetime import datetime
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

### Importando o Dataset

In [2]:
dataset_folder = 'KaggleDatasets/RAW/'
preprocess_folder = 'KaggleDatasets/PRE/'
filename = '201702.csv'
dataset = pd.read_csv(dataset_folder + filename)
dataset.head()

,id,date,input_0,input_1,input_2,input_3,input_4_1,input_5_1,input_6_1,input_7_1,...,output_7_6,output_8_6,output_9_6,output_10_6,output_11_6,output_12_6,output_13_6,output_14_6,output_15_6,output_16_6
0,56644,2017-02-01,4,0,213,62,1.642477,-0.206138,0.547687,1.481902,...,2.629011,4.132933,-0.310499,0.974334,1.343431,0.346723,-0.111801,1.026807,2.651589,5.526916
1,56707,2017-02-01,4,1,213,62,-0.026259,-0.206138,-0.653850,-0.617752,...,-0.387950,-0.614713,-0.310499,-0.765870,-0.685337,-0.993390,-0.111801,-0.319206,-0.223367,-0.379209
2,56770,2017-02-01,4,2,213,62,-0.503290,-0.206138,-0.053082,-0.617752,...,-0.723168,-0.614713,-0.310499,-0.632008,-0.779698,-0.695587,-0.111801,-0.126919,-0.510863,-0.379209
3,56833,2017-02-01,4,3,213,62,-0.651817,-0.206138,-0.653850,-0.617752,...,-0.555559,-0.614713,-0.310499,-0.832801,-0.874059,-0.918939,-0.111801,-0.319206,-0.510863,-0.379209
4,43272,2017-02-01,11,0,185,134,0.379130,-0.247257,0.111462,0.256218,...,3.088040,1.315378,0.814293,0.516683,1.453250,0.529982,-0.165694,0.861235,2.598782,-0.593615


### Removendo Dados

In [3]:
dataset = dataset.drop(['id'], axis=1)

### Convertendo a data para número da semana

In [4]:
dataset['date'] = dataset['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").day)
dataset.head()

,date,input_0,input_1,input_2,input_3,input_4_1,input_5_1,input_6_1,input_7_1,input_8_1,...,output_7_6,output_8_6,output_9_6,output_10_6,output_11_6,output_12_6,output_13_6,output_14_6,output_15_6,output_16_6
0,1,4,0,213,62,1.642477,-0.206138,0.547687,1.481902,-0.518488,...,2.629011,4.132933,-0.310499,0.974334,1.343431,0.346723,-0.111801,1.026807,2.651589,5.526916
1,1,4,1,213,62,-0.026259,-0.206138,-0.653850,-0.617752,-0.518488,...,-0.387950,-0.614713,-0.310499,-0.765870,-0.685337,-0.993390,-0.111801,-0.319206,-0.223367,-0.379209
2,1,4,2,213,62,-0.503290,-0.206138,-0.053082,-0.617752,-0.518488,...,-0.723168,-0.614713,-0.310499,-0.632008,-0.779698,-0.695587,-0.111801,-0.126919,-0.510863,-0.379209
3,1,4,3,213,62,-0.651817,-0.206138,-0.653850,-0.617752,-0.518488,...,-0.555559,-0.614713,-0.310499,-0.832801,-0.874059,-0.918939,-0.111801,-0.319206,-0.510863,-0.379209
4,1,11,0,185,134,0.379130,-0.247257,0.111462,0.256218,-0.614670,...,3.088040,1.315378,0.814293,0.516683,1.453250,0.529982,-0.165694,0.861235,2.598782,-0.593615


### Tratando os NA

In [6]:
# Parâmetros
x = 55 # x%
y = 35 # y%
z = 20 # z%

# Tratamento
# NA > x% : descarta
# x% >NA > y% e variancia < z : descarta
# y% > NA: substitui pela media, moda ou mediana

total = len(dataset)
delete_list = []
median_list = []

variance = dataset.var()
mean = dataset.mean()

for column in dataset:
    num_na = dataset[column].isnull().sum()
    per_na = num_na * 100 / total
    aux = variance[column] * 100 / mean[column]
    if per_na > x:
        delete_list.append(column)
    elif per_na > y and aux > z:
        delete_list.append(column)
    elif num_na > 0:
        median_list.append(column)
        
dataset = dataset.drop(delete_list, axis=1)

median = dataset.median()
for column in median_list:
    dataset[column] = dataset[column].fillna(median[column])
    
dataset.to_csv(preprocess_folder + filename, index=False)